Source: http://www.sthda.com/english/articles/37-model-selection-essentials-in-r/154-stepwise-regression-essentials-in-r/

**To run the R codes on Jupyter notebook, type “conda install -c r r-essentials” on your terminal – it will install the R kernel and some important R packages (e.g. dplyr, ggplot2, etc.)**


In [1]:
#Load the required packages 
library(tidyverse)
library(caret) 
library(leaps)

Warning message:
"package 'tidyverse' was built under R version 3.6.1"-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.1     v purrr   0.3.2
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   0.8.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'ggplot2' was built under R version 3.6.1"Warning message:
"package 'tidyr' was built under R version 3.6.1"Warning message:
"package 'readr' was built under R version 3.6.1"Warning message:
"package 'dplyr' was built under R version 3.6.1"Warning message:
"package 'stringr' was built under R version 3.6.1"Warning message:
"package 'forcats' was built under R version 3.6.1"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'caret' was built under R version 3.6.1"Loading required package: lattice

Attaching package: 'caret'

The following ob

In [2]:
#Load the dataset

df_housing <- read.csv("label_df.csv", stringsAsFactors = FALSE)

In [3]:
#Get an overview of the train data set
#Click on white space below to expand out the code output

glimpse(df_housing)

Observations: 1,460
Variables: 62
$ X             <int> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,...
$ SalePrice     <dbl> 12.24769, 12.10901, 12.31717, 11.84940, 12.42922, 11....
$ LotArea       <dbl> -0.20714171, -0.09188637, 0.07347998, -0.09689747, 0....
$ OverallQual   <dbl> 0.65147924, -0.07183611, 0.65147924, 0.65147924, 1.37...
$ OverallCond   <dbl> -0.5171998, 2.1796278, -0.5171998, -0.5171998, -0.517...
$ YearBuilt     <dbl> 1.05099379, 0.15673371, 0.98475230, -1.86363165, 0.95...
$ YearRemodAdd  <dbl> 0.8786681, -0.4295770, 0.8302146, -0.7202981, 0.73330...
$ MasVnrArea    <dbl> 0.5141039, -0.5707501, 0.3259149, -0.5707501, 1.36648...
$ BsmtFinSF1    <dbl> 0.57542484, 1.17199212, 0.09290718, -0.49927358, 0.46...
$ BsmtUnfSF     <dbl> -0.94459061, -0.64122799, -0.30164298, -0.06166957, -...
$ TotalBsmtSF   <dbl> -0.459302541, 0.466464916, -0.313368755, -0.687324082...
$ X1stFlrSF     <dbl> -0.79343379, 0.25714043, -0.62782603, -0.52173356, -0...
$ X2ndFlrSF     <d

# Method 1: Using stepAIC()
Selects the best model by AIC (Akaike Information Criterion). 

In [6]:
library(MASS)

#Fit your initial model to begin the regression 

full_model <- lm(SalePrice ~ ., data = df_housing)

There are three ways to do stepwise regression: (1) Backward, (2) Forward, (3) Stepwise. To select a particular model, you have to change the value in **direction**: (i) "both" (stepwise regression), (ii) "backward" (for backward regression) and "forward" (for forward selection). 

**trace** can be either TRUE or FALSE. TRUE means you want to see the results at each iteration, while FALSE does not show you each step, you just get the best final model at the end. If you have more variables then TRUE will give you a pretty long list of steps taken. 

In [7]:
stepwise_model <- stepAIC(full_model, direction = "both", trace = FALSE)

# backwards_model <- stepAIC(SalePrice ~ ., direction = "backward", trace = FALSE)

# forwards_model <- stepAIC(SalePrice ~ 1, direction = "forward", scope=formula(full_model), trace = FALSE)

In [8]:
# Get the results of the final model 

summary(stepwise_model)


Call:
lm(formula = SalePrice ~ LotArea + OverallQual + OverallCond + 
    YearBuilt + YearRemodAdd + BsmtFinSF1 + BsmtUnfSF + TotalBsmtSF + 
    X1stFlrSF + GrLivArea + BsmtFullBath + FullBath + HalfBath + 
    KitchenAbvGr + TotRmsAbvGrd + Fireplaces + GarageCars + WoodDeckSF + 
    EnclosedPorch + ScreenPorch + MSZoning + LotShape + LandContour + 
    Neighborhood + BldgType + HouseStyle + Exterior1st + Exterior2nd + 
    ExterCond + Foundation + BsmtQual + BsmtExposure + BsmtFinType1 + 
    BsmtFinType2 + HeatingQC + CentralAir + KitchenQual + Functional + 
    GarageType + PavedDrive + SaleCondition, data = df_housing)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.93211 -0.06439  0.00274  0.07161  0.56042 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   11.8384819  0.0613388 193.002  < 2e-16 ***
LotArea        0.0165700  0.0041406   4.002 6.61e-05 ***
OverallQual    0.1005535  0.0069327  14.504  < 2e-16 ***
OverallCond    0.0455

# Method 2: Using regsubsets()
Has tuning parameter **nvmax** specifying the maximal number of predictors to incorporate in the model. It returns multiple models with different size up to nvmax. You need to compare the performance of the different models for choosing the best one. 

regsubsets() has the option method, which can take the values “backward”, “forward” and “seqrep” (seqrep = sequential replacement, combination of forward and backward selections).

In [5]:
models <- regsubsets(SalePrice ~ ., data = df_housing, nvmax = 10, method = 'seqrep')

summary(models)

Subset selection object
Call: regsubsets.formula(SalePrice ~ ., data = df_housing, nvmax = 10, 
    method = "seqrep")
61 Variables  (and intercept)
              Forced in Forced out
X                 FALSE      FALSE
LotArea           FALSE      FALSE
OverallQual       FALSE      FALSE
OverallCond       FALSE      FALSE
YearBuilt         FALSE      FALSE
YearRemodAdd      FALSE      FALSE
MasVnrArea        FALSE      FALSE
BsmtFinSF1        FALSE      FALSE
BsmtUnfSF         FALSE      FALSE
TotalBsmtSF       FALSE      FALSE
X1stFlrSF         FALSE      FALSE
X2ndFlrSF         FALSE      FALSE
GrLivArea         FALSE      FALSE
BsmtFullBath      FALSE      FALSE
FullBath          FALSE      FALSE
HalfBath          FALSE      FALSE
BedroomAbvGr      FALSE      FALSE
KitchenAbvGr      FALSE      FALSE
TotRmsAbvGrd      FALSE      FALSE
Fireplaces        FALSE      FALSE
GarageCars        FALSE      FALSE
GarageArea        FALSE      FALSE
WoodDeckSF        FALSE      FALSE
OpenPorchSF

# Method 3: CV and GridSearch

The train() function [caret package] provides an easy workflow to perform stepwise selections using the leaps and the MASS packages. It has an option named method, which can take the following values:

 - "leapBackward", to fit linear regression with backward selection
 - "leapForward", to fit linear regression with forward selection
 - "leapSeq", to fit linear regression with stepwise selection
 
You also need to specify the tuning parameter nvmax, which corresponds to the maximum number of predictors to be incorporated in the model.

For example, you can vary nvmax from 1 to 5. In this case, the function starts by searching different best models of different size, up to the best 5-variables model. That is, it searches the best 1-variable model, the best 2-variables model, …, the best 5-variables models.


We can use 5/10-fold cross-validation to estimate the average prediction error (RMSE) of each of the models. The RMSE statistical metric is used to compare the models and to automatically choose the best one, where best is defined as the model that minimize the RMSE.

In [9]:
#Set seed for reproducibility
set.seed(42)

#Set up repeated k-fold cross-validation, indicate the number of folds you want
train_control <- trainControl(method = "cv", number = 5)

#Train the model, indicate the method of regression and range of nvmax numbers to try
step_model <- train(SalePrice ~ ., data = df_housing, 
                   method = "leapSeq",
                   tuneGrid = data.frame(nvmax = 10:200),
                   trControl = train_control)
step_model$results

Warning message:
"predictions failed for Fold1: nvmax=200 Error in method$predict(modelFit = modelFit, newdata = newdata, submodels = param) : 
  Some values of 'nvmax' are not in the model sequence.
"Warning message:
"predictions failed for Fold2: nvmax=200 Error in method$predict(modelFit = modelFit, newdata = newdata, submodels = param) : 
  Some values of 'nvmax' are not in the model sequence.
"Warning message:
"predictions failed for Fold3: nvmax=200 Error in method$predict(modelFit = modelFit, newdata = newdata, submodels = param) : 
  Some values of 'nvmax' are not in the model sequence.
"Warning message:
"predictions failed for Fold4: nvmax=200 Error in method$predict(modelFit = modelFit, newdata = newdata, submodels = param) : 
  Some values of 'nvmax' are not in the model sequence.
"Warning message:
"predictions failed for Fold5: nvmax=200 Error in method$predict(modelFit = modelFit, newdata = newdata, submodels = param) : 
  Some values of 'nvmax' are not in the model sequen

Something is wrong; all the RMSE metric values are missing:
      RMSE        Rsquared        MAE     
 Min.   : NA   Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA   Max.   : NA  
 NA's   :191   NA's   :191   NA's   :191  


ERROR: Error: Stopping


**nvmax**: the number of variable in the model. For example nvmax = 2, specify the best 2-variables model

**RMSE** and **MAE** are two different metrics measuring the prediction error of each model. The lower the RMSE and MAE, the better the model.

**Rsquared** indicates the correlation between the observed outcome values and the values predicted by the model. The higher the R squared, the better the model.

In [10]:
#Display the best tuning values (nvmax) selected by the train() function

step_model$bestTune

,nvmax
6,10
